In [ ]:
%matplotlib inline




Basic MAP Estimation
====================

.. currentmodule:: pydygp.linlatentforcemodels

This note descibes how to simulate observations from the MLFM model, as
well as the process of carrying out MAP parameter estimation for the MLFM
using the Adaptive Gradient matching approximation. This uses the
:class:`MLFMAdapGrad` object and so our first step is to import this object.




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydygp.linlatentforcemodels import MLFMAdapGrad
from sklearn.gaussian_process.kernels import RBF
np.random.seed(17)

Model Setup
~~~~~~~~~~~

To begin we are going to demonstate the MLFM by simulating an ODE on the unit
sphere

\begin{align}S^{2} = \{ x \in \mathbb{R}^3 \; : \; \| x \| = 1 \}.\end{align}

The model will be given by the initial value problem

\begin{align}\dot{\mathbf{x}}(t) = \mathbf{A}(t) \mathbf{x}(t),
    \qquad \mathbf{x}_0 \in S^2,\end{align}

where the coefficient matrix, $\mathbf{A}(t)$, is supported on the Lie
algebra $\mathfrak{so}(3)$. We do this by chosing a basis for the Lie
algebra and then representing each of the structure matrices as a linear combination
of these basis matrices using the coefficients $\beta_{rd}$ leading to a
representation of the coefficient matrix

\begin{align}\mathbf{A}(t) = \sum_{d=0}^3 \beta_{0d}\mathbf{L}_d +
    \sum_{r=1}^R g_r(t) \sum_{d=1}^3 \beta_{rd}\mathbf{L}_d,\end{align}

where $\{\mathbf{L}_d \}$ is a basis of the Lie algebra
$\mathfrak{so}(3)$. And the collection $\{ g_r(t) \}_{r=1}^{R}$
are a set of smooth Gaussian processes. To construct this model in the
:py:obj:`pydygp` package we provide the :class:`pydygp.liealgebras.so` class
which can be used to return basis elements of the Lie algebras of the special
orthogonal group $SO(n)$. For example if we import and call
:py:obj:`so(3)` we get the following output



In [ ]:
from pydygp.liealgebras import so
for d, item in enumerate(so(3)):
    print(''.join(('\n', 'L{}'.format(d+1))))
    print(item)

Having imported the basis matrices for the Lie algebra we also need to
choose how many latent forces we want the model to have, for this example
we are going to consider a single latent forces with RBF kernel. We can
then construct the :class:`MLFMAdapGrad` object



In [ ]:
# Tuple of kernel objects for the latent forces, for r=1,..., R
lf_kernels = (RBF(), )

# construct the MLFM object
mlfm = MLFMAdapGrad(so(3), R=1, lf_kernels=lf_kernels)

Simulation
~~~~~~~~~~
To simulate from the model we need to chose the set of coefficients
$\beta_{r, d}$. We will consider the model with a single latent
forcing function, and randomly generate the variables $beta$.

For this example we are going to consider the case with a known latent
force function given by

\begin{align}g_1(t) = \cos(t) e^{-(t-2)^2}\end{align}

which is the modulation of a cosine signal by a RBF. To simulate we
must pass the initial conditions :py:obj:`x0`, the time points
:py:obj`tt` and the parameters :py:obj:`beta` to the
:func:`MLFMAdapGrad.sim` method. We can
also optionally supply our known latent forces through the
:code:`latent_forces` key word argument which accepts a list, or tuple,
of length R.

The method uses the :class:`odeint` class in scipy to perform the
numerical simulation of the model, and so we can demonstrate they
give equivalent results by also getting the solution using scipy

We also draw attention to the return type of :func:`MLFMAdapGrad.sim`
which when the key word is supplied as :code:`size = M`
with :code:`M > 2` will result in the simulated observations being
returned as a tuple $(Y^{(1)},\ldots,Y^{(M)})$ where
each $Y^{(i)}$ is an $N \times K$ array.



In [ ]:
# Construct the known latent force 
g = lambda t: np.exp(-(t-2)**2) * np.cos(t)  # single latent force

# Simulate the R x D matrix beta
beta = np.random.randn(2, 3)  

# Initial conditions are given by :math:`\mathbf{e}_i` for
# :math:`i=1, 2, 3`.
x0 = np.eye(3)

# The structure matrices as linear combinations of the basis matrices
A = [sum(brd*Ld for brd, Ld in zip(br, so(3)))
     for br in beta]

# dense time vector 
ttd = np.linspace(0., 5., 100)

# solution using Scipy odeint
from scipy.integrate import odeint
scipy_sol = odeint(lambda x, t: (A[0] + g(t)*A[1]).dot(x),
                   x0[0, :],
                   ttd)

# solution using MLFMAdapGrad.sim 

# downsample
tt = ttd[::10]
Data, _ = mlfm.sim(x0, tt, beta=beta, latent_forces=(g, ), size=3)

fig, ax = plt.subplots()
for xk, yk in zip(scipy_sol.T[:-1], Data[0].T[:-1]):
    ax.plot(ttd, xk, 'k-', alpha=0.3)
    ax.plot(tt, yk, 'C0o')
ax.plot(ttd, scipy_sol[:, -1], 'k-', alpha=0.3, label='scipy odeint')
ax.plot(tt, Data[0][:, -1], 'C0o', label='MLFMAdapGrad.sim')
ax.set_xlabel('Time')
ax.legend()

Latent Force Estimation
~~~~~~~~~~~~~~~~~~~~~~~



In [ ]:
Y = np.column_stack(y.T.ravel() for y in Data)
res = mlfm.fit(tt, Y, beta0 = beta, beta_is_fixed=True)

# predict the lf using the Laplace approximation
Eg, SDg = mlfm.predict_lf(ttd, return_std=True)

# sphinx_gallery_thumbnail_number = 2
fig2, ax = plt.subplots()
ax.plot(ttd, g(ttd), 'k-', alpha=0.8)
ax.plot(tt, res.g.T, 'o')
for Egr, SDgr in zip(Eg, SDg):
    ax.fill_between(ttd,
                    Egr + 2*SDgr, Egr - 2*SDgr,
                    alpha=0.5)

plt.show()